In [1]:
import pandas as pd
npr = pd.read_csv('npr.csv')
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Article'])

In [3]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [5]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [6]:
len(cv.get_stop_words())

318

In [7]:
len(LDA.components_)

7

In [8]:
LDA.components_[0]

array([8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
       1.43006821e-01, 1.42902042e-01, 1.42861626e-01])

In [9]:
type(npr)

pandas.core.frame.DataFrame

In [10]:
LDA.components_.sum(axis=1)

array([533709.91472191, 664554.15090382, 716452.68503894, 634031.81220884,
       779343.7355011 , 821109.26801128, 426562.43356143])

In [11]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 20 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print('\n')

THE TOP 20 WORDS FOR TOPIC #0
['president', 'state', 'tax', 'insurance', 'trump', 'companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 20 WORDS FOR TOPIC #1
['white', 'according', 'attack', 'reported', 'war', 'military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 20 WORDS FOR TOPIC #2
['little', 'know', 'don', 'year', 'make', 'way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 20 WORDS FOR TOPIC #3
['world', 'research', 'university', 'percent', 'care', 'time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 20 WORDS FOR TOPIC #4
['donald', 'political', 'states', 'law', 'just', 'voters', 'vote', 'election', 'par

In [12]:
topic_results = LDA.transform(dtm)

In [13]:
topic_results.shape

(11992, 7)

In [14]:
topic_results[5].sum(axis=0)

1.0

In [15]:
type(topic_results)

numpy.ndarray

In [16]:
import numpy as np
def jsd(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    x = np.array(x)
    y = np.array(y)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)    
    return d



In [17]:
jsd(topic_results[2],topic_results[11968])

0.05506031357963158

In [18]:
def taking_input():
    article_no = 0
    
    article_no  = int(input('plz mention the ID of the article to get similar articles --  '))
    if article_no not in range(11992):
        print ('plz give the correct input')
    else:
        return article_no

In [19]:
def similar_article(article_no):
    dictionary = {}
    for i in range(11992):
       
        if i == article_no:
            continue
        elif jsd(topic_results[article_no],topic_results[i]) < 0.1:
           
            dictionary[jsd(topic_results[article_no],topic_results[i])] = i
    return(sorted(dictionary.items())[0:10])
            

In [23]:
def replay():
    
    print('\n')
    return input('Do you want to find any other similar articles Enter Yes or No: ').lower().startswith('y')

In [25]:
print("lets find the most similar article")
print("\n")
print("\n")


while True:
    article_input = taking_input()
    result = similar_article(article_input) 
    print (result, end="\n")
    
    if not replay():
        break

lets find the most similar article




plz mention the ID of the article to get similar articles --  5000
[(0.010218640793032198, 6044), (0.01787774514430938, 6239), (0.01979721953490365, 9540), (0.021301002768229816, 11895), (0.02375669183675107, 1097), (0.032487809851030905, 5826), (0.036954616363381675, 1260), (0.04480496786200738, 2774), (0.04611237535315601, 7816), (0.05085615138921963, 4165)]


Do you want to find any other similar articles Enter Yes or No: yes
plz mention the ID of the article to get similar articles --  10000
[(0.0025142650252498104, 7453), (0.002521918820398787, 1230), (0.006540844993002438, 2972), (0.007063580325759817, 4636), (0.007080134428572707, 4298), (0.007486464043699997, 1947), (0.008630878162003412, 9469), (0.008774819456437516, 4740), (0.008885885894286183, 5274), (0.008926256669472617, 5948)]


Do you want to find any other similar articles Enter Yes or No: no
